In [4]:
from datasets import load_dataset, ReadInstruction
import spacy
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from transformers import AutoTokenizer
import numpy as np
# notebook tqdm
from tqdm.notebook import tqdm

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
# df_deduped_pythia = pd.read_pickle("../data/deduped_pythia.pkl")

In [6]:
df_duped_pythia = pd.read_pickle("../data/duped_pythia.pkl")

In [7]:
def get_avg_dependency_length(sentence):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sentence)
    ls_lengths = []
    for token in doc:
        for child in token.children:
            distance = child.i - token.i
            # print(f"{token.text} --{child.dep_}--> {child.text}  (distance: {distance})")
            ls_lengths.append(abs(distance))
    return np.mean(ls_lengths)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m-deduped")

def get_string(token_indices):
    # token_indices = list(token_indices)
    return tokenizer.decode(token_indices)

In [8]:
# df_deduped_pythia['first_64_tokens'] = df_deduped_pythia['tokens'].parallel_apply(lambda x: x[:64])
# df_deduped_pythia['first_64_tokens_string'] = df_deduped_pythia['first_64_tokens'].parallel_apply(get_string)
# deduped_value_counts = df_deduped_pythia['is_memorized'].value_counts()
# min_count = deduped_value_counts.min()
# df_deduped_pythia_min = df_deduped_pythia.groupby('is_memorized').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
# ls_first_64_tokens_string = df_deduped_pythia_min['first_64_tokens_string'].to_list()

# from concurrent.futures import ThreadPoolExecutor, as_completed
# import os
# ls_dependency_lengths = []
# # use a thread pool to parallelize the computation of dependency lengths
# with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
#     futures = [executor.submit(get_avg_dependency_length, sent) for sent in ls_first_64_tokens_string]

#     # use tqdm to display a progress bar while the threads are running
#     for future in tqdm(as_completed(futures), total=len(futures)):
#         ls_dependency_lengths.append(future.result())

# # print the list of average dependency lengths for each sentence
# print(ls_dependency_lengths)

# import pickle
# # save the list of average dependency lengths for each sentence
# with open("../data/ls_dependency_lengths.pkl", "wb") as f:
#     pickle.dump(ls_dependency_lengths, f)

# df_deduped_pythia_min['dependency_length'] = ls_dependency_lengths

# # mean dependency length for memorized sentences based on is_memorized
# df_deduped_pythia_min.groupby('is_memorized')['dependency_length'].mean()

# df_deduped_pythia_min.to_pickle("../data/df_deduped_pythia_min_with_dependency_length.pkl")

In [9]:
df_duped_pythia['first_64_tokens'] = df_duped_pythia['tokens'].parallel_apply(lambda x: x[:64])
df_duped_pythia['first_64_tokens_string'] = df_duped_pythia['first_64_tokens'].parallel_apply(get_string)
duped_value_counts = df_duped_pythia['is_memorized'].value_counts()
min_count = duped_value_counts.min()
print(duped_value_counts)
df_duped_pythia_min = df_duped_pythia.groupby('is_memorized').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
ls_first_64_tokens_string = df_duped_pythia_min['first_64_tokens_string'].to_list()

from concurrent.futures import ThreadPoolExecutor, as_completed
import os
ls_dependency_lengths_duped = []
# use a thread pool to parallelize the computation of dependency lengths
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = [executor.submit(get_avg_dependency_length, sent) for sent in ls_first_64_tokens_string]

    # use tqdm to display a progress bar while the threads are running
    for future in tqdm(as_completed(futures), total=len(futures)):
        ls_dependency_lengths_duped.append(future.result())

# print the list of average dependency lengths for each sentence
print(ls_dependency_lengths_duped)

import pickle
# save the list of average dependency lengths for each sentence
with open("../data/ls_dependency_lengths_duped.pkl", "wb") as f:
    pickle.dump(ls_dependency_lengths_duped, f)

df_duped_pythia_min['dependency_length'] = ls_dependency_lengths_duped

# mean dependency length for memorized sentences based on is_memorized
print(df_duped_pythia_min.groupby('is_memorized')['dependency_length'].mean())

df_duped_pythia_min.to_pickle("../data/df_duped_pythia_min_with_dependency_length.pkl")

is_memorized
False    490591
True       9409
Name: count, dtype: int64


  0%|          | 0/18818 [00:00<?, ?it/s]

/home/aflah/anaconda3/envs/sem_mem/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/aflah/anaconda3/envs/sem_mem/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[3.3333333333333335, 2.9454545454545453, 3.7115384615384617, 2.7719298245614037, 4.237288135593221, 2.740740740740741, 2.5454545454545454, 2.5833333333333335, 2.7254901960784315, 4.609375, 2.6153846153846154, 3.925925925925926, 3.84, 4.6521739130434785, 3.193548387096774, 3.2, 3.830508474576271, 2.9473684210526314, 3.706896551724138, 3.5294117647058822, 2.926829268292683, 5.730769230769231, 4.558139534883721, 2.7142857142857144, 2.4583333333333335, 3.25, 2.618181818181818, 2.7941176470588234, 2.9622641509433962, 3.152542372881356, 2.9, 3.225806451612903, 3.0588235294117645, 2.4833333333333334, 3.088888888888889, 2.9, 4.618181818181818, 4.62962962962963, 3.6666666666666665, 4.96875, 3.347826086956522, 3.23728813559322, 3.0, 2.3636363636363638, 2.097560975609756, 4.046875, 20.94736842105263, 3.2666666666666666, 4.0181818181818185, 3.2916666666666665, 2.9464285714285716, 3.68, 3.8035714285714284, 2.824561403508772, 2.8529411764705883, 3.8666666666666667, 3.0, 2.509433962264151, 3.66071428